In [6]:
from neuron import h
import os
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))




#First compile our modfiles.
if os.system('nrnivmodl modfiles/') == 0:
    print("Compilation Succeeded.")
else:
    print("Compilation Failed.")

Compilation Succeeded.


In [7]:
h.load_file('stdrun.hoc')

#Define simumulation parameters.
SIMRUNTIME = 1500
DELAY      = 400
INJTIME    = 500
I_INJ      = 0.1

h.tstop    = SIMRUNTIME
h.v_init   = -75

#Now set up vectors to record the membrane potential and the time.
potential = h.Vector()
time      = h.Vector()

In [13]:
import numpy as np

tonic_soma = h.Section(name = 'tonic_soma')

tonic_iclamp = h.IClamp(tonic_soma(0.5))
tonic_iclamp.dur   = INJTIME
tonic_iclamp.amp   = I_INJ
tonic_iclamp.delay = DELAY

#We will make a super simple one compartement tonic spiking cell.
tonic_soma.nseg = 1
tonic_soma.L    = 25
tonic_soma.diam = 24.75

#Now insert the two segregated modules into the cell.
#First insert the passive module.
tonic_soma.insert('leakCA3')
tonic_soma.insert('hdCA3')

#Now insert the spiking module.
tonic_soma.insert('natCA3')
tonic_soma.insert('kdrCA3')

#record time and membrane potential for tonic cell.
#Record the membrane variables.
time.record(h._ref_t) 
potential.record(tonic_soma(0.5)._ref_v)


def run_tonic_model(amp, gleak, gna, gkdr, gh, tau_m, tau_h, tau_n, tau_l, vh_m, vh_h, vh_n, vh_l):

    tonic_iclamp.amp = amp

    tonic_soma.glbar_leakCA3   = gleak #1.53*3e-5
    tonic_soma.gbar_natCA3     = gna
    tonic_soma.mtfactor_natCA3 = tau_m
    tonic_soma.htfactor_natCA3 = tau_h
    tonic_soma.vhalfm_natCA3   = vh_m
    tonic_soma.vhalfh_natCA3   = vh_h
    tonic_soma.gbar_kdrCA3     = gkdr
    tonic_soma.ntfactor_kdrCA3 = tau_n
    tonic_soma.vhalfn_kdrCA3   = vh_n
    tonic_soma.ghdbar_hdCA3    = gh
    tonic_soma.ltfactor_hdCA3  = tau_l
    tonic_soma.vhalfl_hdCA3    = vh_l
    
    h.run()

#Set the default parameters for the simulations.

default_tonic = [I_INJ, 1.53*3e-5, 0.065, 0.065, 1.0e-5, 
                        1.0      , 1.0  , 1.0  , 1.0   ,
                        -35.5    , -40  , 0.0  , -75.0  ]

run_tonic_model(default_tonic[0], default_tonic[1], default_tonic[2] ,  default_tonic[3],  default_tonic[4],
                                  default_tonic[5], default_tonic[6] ,  default_tonic[7],  default_tonic[8],
                                  default_tonic[9], default_tonic[10], default_tonic[11], default_tonic[12] )


In [29]:
#Now lets implement the SBI side of things.
import sbi.utils as utils
from sbi.inference.base import infer
import torch
from scipy.signal import find_peaks

#Given the votage trace provided in the potential vector we can calculate the summary statistics of the spiker.
#The statistics for this model will be spike number, resting potential, average spike height, and average spike width.
def calculate_summary_statistics():
    trace = potential.as_numpy()

    zero_crossings = np.where(np.diff(np.sign(trace)))[0] #Detect when the voltage trace crosses 0.
    
    spike_width = 0
    for i in range(0,len(zero_crossings), 2):
        spike_width += zero_crossings[i + 1] - zero_crossings[i]
    
    spike_num = (len(zero_crossings) / 2)
    
    if spike_num != 0:
        spike_width /= spike_num
    
    resting = trace[-1]
    
    peaks, _ =  find_peaks(trace,0)
    
    #Calculate average height.
    average_spike_height = np.mean(_['peak_heights'])
    
    return [spike_num, resting, average_spike_height, spike_width]

#Now define the simulation wrapper.
def simulation_wrapper(params):
    run_tonic_model(params[0], params[1], params[2] ,  params[3],  params[4],
                               params[5], params[6] ,  params[7],  params[8],
                               params[9], params[10], params[11], params[12] )
    sum_stats = torch.as_tensor(calculate_summary_statistics(), dtype=torch.float)
    
    return sum_stats

#Now define min and max parameter values.
min_values = [0.1, 1.53*3e-5, 0.05, 0.05, 1.0e-5, 
            1.0      , 1.0  , 1.0  , 1.0   ,
            -35.5    , -40  , 0.0  , -75.0  ]

max_values = [0.1, 1.53*3e-5, 0.07, 0.07, 1.0e-5, 
            1.0      , 1.0  , 1.0  , 1.0   ,
            -35.5    , -40  , 0.0  , -75.0  ]

#Set up ranges for inference.
prior = utils.torchutils.BoxUniform(low  = torch.as_tensor(min_values),
                                    high = torch.as_tensor(max_values))

#Run the inference simulator.
#THERE IS A PROBLEM WITH PICKLE HERE, NUM_WORKERS NEEDS TO BE 1
#Check: https://www.mackelab.org/sbi/faq/question_03/
#for solution
posterior = infer(simulation_wrapper, prior, method='SNPE', 
                  num_simulations=300, num_workers=1)


/usr/neurotools/conda/envs/py36/lib/python3.6/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


Neural network successfully converged after 435 epochs.


/usr/neurotools/conda/envs/py36/lib/python3.6/site-packages/sbi/utils/plot.py:567: UserWarning: Attempting to set identical left == right == 0.10000000149011612 results in singular transformations; automatically expanding.
  ax.set_xlim((limits[col][0], limits[col][1]))
/usr/neurotools/conda/envs/py36/lib/python3.6/site-packages/sbi/utils/plot.py:567: UserWarning: Attempting to set identical left == right == 4.5900000259280205e-05 results in singular transformations; automatically expanding.
  ax.set_xlim((limits[col][0], limits[col][1]))
/usr/neurotools/conda/envs/py36/lib/python3.6/site-packages/sbi/utils/plot.py:567: UserWarning: Attempting to set identical left == right == 9.999999747378752e-06 results in singular transformations; automatically expanding.
  ax.set_xlim((limits[col][0], limits[col][1]))
/usr/neurotools/conda/envs/py36/lib/python3.6/site-packages/sbi/utils/plot.py:567: UserWarning: Attempting to set identical left == right == 1.0 results in singular transformations; 